# Comparative Analysis of Proteins in the Mouse Brain
# Building a Database
Authors:
1. Defteraiou Maria (7115152400003)
2. Konstantinidis Konstantinos (7115152400017)
3. Mertzani Styliani (7115152400033)
4. Voulgari Despoina (7115152400012)

Subject: **Intro to Biotechnology**

Professor: **Tsangaris G.**

In [1]:
import pandas as pd
from functools import reduce
from IPython.display import display
import sys
import requests
from matplotlib import pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import os
import time
import glob

## Refactoring the Data 

In [2]:
# Load excel files and convert them to data frames
cerebellum_df = pd.read_excel("data/Mouse_Brain_-_Cerebellum_7_weeks.xlsx")
cerebellum_df.rename(columns={"Accession": "Uniprot_ID"}, inplace=True)

cortex_df = pd.read_excel("data/Mouse_Brain_-_Cortex_7_weeks.xlsx")
cortex_df.rename(columns={"Accession": "Uniprot_ID"}, inplace=True)

mid_brain_df = pd.read_excel("data/Mouse_Brain_-_Mid_Brain_7_weeks.xlsx")
mid_brain_df.rename(columns={"Accession": "Uniprot_ID"}, inplace=True)

hipocampus_df = pd.read_excel("data/mouse_brain_-_hipocampus_7_weeks-01.xlsx")
hipocampus_df.rename(columns={"Accession": "Uniprot_ID"}, inplace=True)

hipothalamus_df = pd.read_excel("data/mouse_brain_-_hipothalamus_7_weeks-01.xlsx")
hipothalamus_df.rename(columns={"Accession": "Uniprot_ID"}, inplace=True)

medulla_df = pd.read_excel("data/mouse_brain_-_medulla_7_weeks_old-01.xlsx")
medulla_df.rename(columns={"Accession": "Uniprot_ID"}, inplace=True)

olfactory_df = pd.read_excel("data/mouse_brain_-_olfactory_balb_7_weeks-01.xlsx")
olfactory_df.rename(columns={"Accession": "Uniprot_ID"}, inplace=True)


supplementary_df = pd.read_excel("data/Supplementary Table 8.xlsx")
supplementary_df.rename(columns={"Entry": "Uniprot_ID"}, inplace=True)

taraslia_1_df = pd.read_excel("data/TARASLIA et al TABLE 1.xls", skiprows=1)
taraslia_1_df.rename(columns={"Accession Name": "Uniprot_ID"}, inplace=True)

taraslia_2_df = pd.read_excel("data/TARASLIA et al TABLE 2.xls", skiprows=1)
taraslia_2_df.rename(columns={"Accession Name": "Uniprot_ID"}, inplace=True)

taraslia_3_df = pd.read_excel("data/TARASLIA et al TABLE 3.xls", skiprows=1)
taraslia_3_df.rename(columns={"Accession Name": "Uniprot_ID"}, inplace=True)

taraslia_4_df = pd.read_excel("data/TARASLIA et al TABLE 4.xls", skiprows=1)
taraslia_4_df.rename(columns={"Accession Name": "Uniprot_ID"}, inplace=True)

taraslia_5_df = pd.read_excel("data/TARASLIA et al TABLE 5.xls", skiprows=1)
taraslia_5_df.rename(columns={"Accession Name": "Uniprot_ID"}, inplace=True)

taraslia_6_df = pd.read_excel("data/TARASLIA et al TABLE 6.xls", skiprows=1)
taraslia_6_df.rename(columns={"Accession Name": "Uniprot_ID"}, inplace=True)

taraslia_7_df = pd.read_excel("data/TARASLIA et al TABLE 7.xls", skiprows=1)
taraslia_7_df.rename(columns={"Accession Name": "Uniprot_ID"}, inplace=True)

taraslia_8_df = pd.read_excel("data/TARASLIA et al TABLE 8.xls", skiprows=1)
taraslia_8_df.rename(columns={"Accession Name": "Uniprot_ID"}, inplace=True)

In [3]:
for i in range(1, 9):
    first_row = pd.read_excel(f"data/TARASLIA et al TABLE {i}.xls", header=None).iloc[0, 0]
    if i == 1:
        display(f"{first_row.split(".")[0]} ~> {first_row.split("in the ")[-1].replace(".", "")}")
    else:
        display(f"{first_row.split(".")[0]} ~> {first_row.split("only in ")[-1]}")

'Table I ~> nornal mouse brain parts'

'Table II ~> olfactory bulb'

'Table III ~> hippocampus '

'Table IV ~> midbrain '

'Table V ~> medulla '

'Table VI ~> cerebellum '

'Table VII ~> hypothalamus '

'Table VIII ~> cortex'

In [4]:
print("Entries in each file:")

print(f"  CB new: {cerebellum_df.shape[0]}")
print(f"  CC new: {cortex_df.shape[0]}")
print(f"  MB new: {mid_brain_df.shape[0]}")
print(f"  HC new: {hipocampus_df.shape[0]}")
print(f"  HT new: {hipothalamus_df.shape[0]}")
print(f"  MD new: {medulla_df.shape[0]}")
print(f"  OB new: {olfactory_df.shape[0]}")

print(f"  Supplementary: {supplementary_df.shape[0]}")

print(f"  All old (taraslia 1): {taraslia_1_df.shape[0]}")
print(f"  OB old (taraslia 2): {taraslia_2_df.shape[0]}")
print(f"  HC old (taraslia 3): {taraslia_3_df.shape[0]}")
print(f"  MB old (taraslia 4): {taraslia_4_df.shape[0]}")
print(f"  MD old (taraslia 5): {taraslia_5_df.shape[0]}")
print(f"  CB old (taraslia 6): {taraslia_6_df.shape[0]}")
print(f"  HT old (taraslia 7): {taraslia_7_df.shape[0]}")
print(f"  CC old (taraslia 8): {taraslia_8_df.shape[0]}")

Entries in each file:
  CB new: 2334
  CC new: 2795
  MB new: 2250
  HC new: 2246
  HT new: 2247
  MD new: 2391
  OB new: 2311
  Supplementary: 6296
  All old (taraslia 1): 515
  OB old (taraslia 2): 54
  HC old (taraslia 3): 62
  MB old (taraslia 4): 9
  MD old (taraslia 5): 10
  CB old (taraslia 6): 5
  HT old (taraslia 7): 8
  CC old (taraslia 8): 36


In [5]:
brain_regions = [
    "CB",
    "CC",
    "MB",
    "HC",
    "HT",
    "MD",
    "OB"
]
all_data = {
    "CB": cerebellum_df,
    "CC": cortex_df,
    "MB": mid_brain_df,
    "HC": hipocampus_df,
    "HT": hipothalamus_df,
    "MD": medulla_df,
    "OF": olfactory_df,

    "SUPPLEMENTARY": supplementary_df,

    "ALL": taraslia_1_df,
    "taraslia_OB": taraslia_2_df,
    "taraslia_HC": taraslia_3_df,
    "taraslia_MB": taraslia_4_df,
    "taraslia_MD": taraslia_5_df,
    "taraslia_CB": taraslia_6_df,
    "taraslia_HT": taraslia_7_df,
    "taraslia_CC": taraslia_8_df    
}
data_new = {
    "CB": cerebellum_df,
    "CC": cortex_df,
    "MB": mid_brain_df,
    "HC": hipocampus_df,
    "HT": hipothalamus_df,
    "MD": medulla_df,
    "OB": olfactory_df  
}
data_old = {
    "ALL": taraslia_1_df,
    "OB": taraslia_2_df,
    "HC": taraslia_3_df,
    "MB": taraslia_4_df,
    "MD": taraslia_5_df,
    "CB": taraslia_6_df,
    "HT": taraslia_7_df,
    "CC": taraslia_8_df  
}

In [6]:
for k, v in all_data.items():
    print(k)
    display(v.head())

CB


,Uniprot_ID,Description,Score,Coverage,# Proteins,# Unique Peptides,# Peptides,# PSMs,Area,# AAs,MW [kDa],calc. pI
0,P16546,"Spectrin alpha chain, non-erythrocytic 1 OS=Mu...",656.608884,42.64,1,80,81,256,2.904653e+08,2472,284.422297,5.325684
1,Q04447,Creatine kinase B-type OS=Mus musculus GN=Ckb ...,299.488654,50.39,1,15,15,112,8.276152e+08,381,42.686280,5.668457
2,Q6PIC6,Sodium/potassium-transporting ATPase subunit a...,270.776774,28.83,3,13,21,106,1.501888e+09,1013,111.619932,5.414551
3,Q6PIE5,Sodium/potassium-transporting ATPase subunit a...,243.238156,28.82,3,15,23,90,6.516163e+08,1020,112.145477,5.554199
4,Q60932,Voltage-dependent anion-selective channel prot...,221.960705,49.32,1,9,10,77,7.135767e+08,296,32.331378,8.426270


CC


,Uniprot_ID,Description,Score,Coverage,# Proteins,# Unique Peptides,# Peptides,# PSMs,Area,# AAs,MW [kDa],calc. pI
0,P16546,"Spectrin alpha chain, non-erythrocytic 1 OS=Mu...",1033.115718,42.96,2,80,80,308,1.534083e+09,2472,284.422297,5.325684
1,Q60932,Voltage-dependent anion-selective channel prot...,500.948720,49.32,1,11,12,191,2.665349e+09,296,32.331378,8.426270
2,Q6PIC6,Sodium/potassium-transporting ATPase subunit a...,478.390221,28.53,2,13,21,151,6.228773e+09,1013,111.619932,5.414551
3,Q62261,"Spectrin beta chain, non-erythrocytic 1 OS=Mus...",354.685655,34.79,1,51,52,114,4.879352e+08,2363,274.052467,5.579590
4,O88935,Synapsin-1 OS=Mus musculus GN=Syn1 PE=1 SV=2 -...,350.253051,39.24,1,15,16,114,2.110694e+09,706,74.051591,9.803223


MB


...

,Uniprot_ID,Protein name,Mascot Score,Coverage (%),Protein MW (Da),pI-Value
0,4EBP2_MOUSE,Eukaryotic translation initiation factor 4E-bi...,55,80,13004,6.10
1,CB070_MOUSE,UPF0573 protein C2orf70 homolog,57,43,23113,10.06
2,CYC_MOUSE,"Cytochrome c, somatic",55,39,11712,10.19
3,ENOB_MOUSE,Beta-enolase,58,33,47337,6.89
4,PHLB2_MOUSE,Pleckstrin homology-like domain family B membe...,57,15,142253,8.26


taraslia_HT


,Uniprot_ID,Protein name,Mascot Score,Coverage (%),Protein MW (Da),pI-Value
0,FBX16_MOUSE,F-box only protein 16,55,35,39362,10.64
1,K1C15_MOUSE,"Keratin, type I cytoskeletal 15",56,25,49278,4.60
2,LAP2B_MOUSE,"Lamina-associated polypeptide 2, isoforms beta...",60,27,50378,9.90
3,MYH1_MOUSE,Myosin-1,57,16,224116,5.50
4,OTUB1_MOUSE,Ubiquitin thioesterase OTUB1,55,36,31478,4.70


taraslia_CC


,Uniprot_ID,Protein name,Mascot Score,Coverage (%),Protein MW (Da),pI-Value
0,ABD12_MOUSE,Monoacylglycerol lipase ABHD12,87,32,45640,9.60
1,AL9A1_MOUSE,4-trimethylaminobutyraldehyde dehydrogenase,82,31,54449,6.77
2,ARK72_MOUSE,Aflatoxin B1 aldehyde reductase member 2,124,48,41028,9.30
3,CN166_MOUSE,UPF0568 protein C14orf166 homolog,116,57,28249,6.50
4,CTBP1_MOUSE,C-terminal-binding protein 1,59,32,48171,6.30


In [7]:
def get_url(url, **kwargs):
    """
    Sends a GET request to the specified URL and handles the response.

    Args:
        url (str): The URL to send the GET request to.
        **kwargs: Additional keyword arguments to pass to `requests.get`.

    Returns:
        requests.Response: The response object if the request is successful.

    Raises:
        HTTPError: If the request fails (non-2xx status code).
        SystemExit: If the request fails, the program exits after printing the error.
    """
    try:
        response = requests.get(url, **kwargs)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

In [8]:
def refactor_code_single(prot_id, website_api="https://rest.uniprot.org"):
    """
    Queries the UniProt API to retrieve the primary accession ID for a given protein ID.

    Args:
        prot_id (str): The UniProt identifier (e.g., "4EBP2_MOUSE" or "P16546").

    Returns:
        tuple: A tuple containing the original protein ID and its corresponding primary accession ID.

    Raises:
        KeyError: If the expected "primaryAccession" field is not found in the response.
        requests.RequestException: If there is an issue with the API request.
    """
    response = get_url(f"{website_api}/uniprotkb/search?query={prot_id}")
    response = response.json()
    return prot_id, response["results"][0]["primaryAccession"]


def refactor_codes(df):
    """
    Updates UniProt IDs in a DataFrame by replacing them with their primary accession IDs.

    Args:
        df (pd.DataFrame): A pandas DataFrame containing a column "Uniprot_ID" with UniProt identifiers.

    Returns:
        pd.DataFrame: The updated DataFrame with UniProt IDs replaced by their primary accession IDs.

    Notes:
        - Uses multithreading (ThreadPoolExecutor) to process UniProt ID lookups in parallel.
        - If a lookup fails, the original ID remains unchanged.
    """
    with ThreadPoolExecutor() as executor:
        future_to_id = {executor.submit(refactor_code_single, prot_id): prot_id for prot_id in df["Uniprot_ID"]}
        for future in as_completed(future_to_id):
            old_id, new_id = future.result()
            df["Uniprot_ID"].replace(old_id, new_id, inplace=True)
    return df


def refactor_wrapper(d):
    """
    Applies UniProt ID updates to multiple DataFrames stored in a dictionary.

    Args:
        d (dict): A dictionary where keys are region names (str) and values are pandas DataFrames.

    Returns:
        dict: The updated dictionary with modified DataFrames where UniProt IDs are replaced.

    Notes:
        - Uses multithreading to process multiple DataFrames in parallel.
        - Each DataFrame is processed using `refactor_codes`.
        - If an update fails, the original DataFrame remains unchanged.
    """
    total = sum([len(df) for k, df in d.items()])
    with ThreadPoolExecutor() as executor:
        future_to_region = {executor.submit(refactor_codes, df): region for region, df in d.items()}
        for future in as_completed(future_to_region):
            region = future_to_region[future]
            d[region] = future.result()
    return d


In [9]:
data_old = refactor_wrapper(data_old)

In [10]:
for k, v in data_old.items():
    print(k)
    display(v.head(3))

ALL


,Uniprot_ID,Protein name,Protein MW,pI-Value,Brain part,Subcellular localisation,Function
0,Q9CQV8,14-3-3 protein beta/alpha,28183,4.62,"OB, HT, MD",Cytoplasm,"Transcription rergulation, cytoplasmic sequest..."
1,P62259,14-3-3 protein epsilon,29326,4.50,"OB, MB, HT, HC, CB, MD",Cytoplasm,"Cerebral cortex development, hippocampus devel..."
2,P68510,14-3-3 protein eta,28365,4.66,OB,Cytoplasm,"Glucocorticoid catabolism, glucocorticoid rece..."


OB


,Uniprot_ID,Protein name,Mascot Score,Coverage (%),Protein MW (Da),pI-Value
0,P68510,14-3-3 protein eta,64,40,28365,4.66
1,Q99L13,"3-hydroxyisobutyrate dehydrogenase, mitochondr...",112,40,35816,9.30
2,Q8BFZ3,Beta-actin-like protein 2,58,22,42319,5.19


HC


...

,Uniprot_ID,Protein name,Mascot Score,Coverage (%),Protein MW (Da),pI-Value
0,P70445,Eukaryotic translation initiation factor 4E-bi...,55,80,13004,6.10
1,Q9DAS2,UPF0573 protein C2orf70 homolog,57,43,23113,10.06
2,P62897,"Cytochrome c, somatic",55,39,11712,10.19


HT


,Uniprot_ID,Protein name,Mascot Score,Coverage (%),Protein MW (Da),pI-Value
0,Q9QZM9,F-box only protein 16,55,35,39362,10.64
1,Q61414,"Keratin, type I cytoskeletal 15",56,25,49278,4.60
2,Q61029,"Lamina-associated polypeptide 2, isoforms beta...",60,27,50378,9.90


CC


,Uniprot_ID,Protein name,Mascot Score,Coverage (%),Protein MW (Da),pI-Value
0,Q99LR1,Monoacylglycerol lipase ABHD12,87,32,45640,9.60
1,Q9JLJ2,4-trimethylaminobutyraldehyde dehydrogenase,82,31,54449,6.77
2,Q8CG76,Aflatoxin B1 aldehyde reductase member 2,124,48,41028,9.30


In [11]:
for k, v in data_old.items():
    v.to_csv(f"refactored_data/old_{k}.csv", index=False)

In [12]:
for k, v in data_new.items():
    v.to_csv(f"refactored_data/new_{k}.csv", index=False)

In [13]:
all_data["SUPPLEMENTARY"].to_csv(f"refactored_data/new_supplementary.csv", index=False)

In [ ]:
for region in brain_regions:
    proteins_old = data_old.to_list()
    print(proteins_old)
    proteins_new = data_new.to_list()
    print(proteins_new)

## Preprocessing the Data

In [ ]:
def get_url(url, timeout=15, retries=10, **kwargs):
    """Fetch API response with retries and error handling."""
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=timeout, **kwargs)
            response.raise_for_status()
            return response.json()

        except requests.exceptions.Timeout:
            print(f"Timeout for URL: {url}")
        except requests.exceptions.ConnectionError:
            print(f"Connection error: {url}")
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")

        time.sleep(1)

    print(f"Skipping {url} after {retries} failed attempts.")
    return None


def fetch_uniprot_data(prot_id, website_api="https://rest.uniprot.org"):
    """Fetch UniProt data once per ID, skip if failing."""
    url = f"{website_api}/uniprotkb/search?query={prot_id}"
    response = get_url(url)

    if response and "results" in response and response["results"]:
        return response["results"][0]
    else:
        print(f"Skipping protein {prot_id} (No valid response)")
        return None


def extract_go_terms(result):
    """Extract GO terms categorized by type."""
    go_res = {"C": [], "F": [], "P": []}
    for ref in result.get("uniProtKBCrossReferences", []):
        if ref["database"] == "GO":
            go_type, go_term = ref["properties"][0]["value"].split(":")
            go_res.setdefault(go_type, []).append(go_term)
    return go_res


def extract_ptm(result):
    """Extract PTM-related keywords."""
    return [kw["name"] for kw in result.get(
        "keywords", []
    ) if kw["category"] == "PTM"]


def extract_go_ids(result):
    """Extract GO IDs from UniProt API result."""
    ids = []
    for ref in result.get("uniProtKBCrossReferences", []):
        if ref["database"] == "GO":
            ids.append(ref["id"])
    return ids


def find_ancestors(go, website_api="https://www.ebi.ac.uk/QuickGO"):
    """Find ancestor GO terms using QuickGO API."""
    url = f"{website_api}/services/ontology/go/terms/{go}/ancestors?relations=is_a"
    response = get_url(url)
    if response and "results" in response and "ancestors" in response["results"][0]:
        return response["results"][0]["ancestors"]
    return []


def find_root_ancestors(response, biological_process_map, molecular_function_map):
    """Map root ancestors to biological process and molecular function."""
    result = {"BiologicalProcess": [], "MolecularFunction": []}

    for go_id in response:
        if go_id in biological_process_map:
            result["BiologicalProcess"].append(biological_process_map[go_id])
        if go_id in molecular_function_map:
            result["MolecularFunction"].append(molecular_function_map[go_id])
    return result


def process_protein(
        protein,
        uniprot_data,
        biological_process_map,
        molecular_function_map
    ):
    """Process a single protein based on pre-fetched UniProt data."""
    result = uniprot_data.get(protein)  # Get pre-fetched data
    
    if not result:  # Skip if UniProt request failed
        return {
            "Uniprot_ID": protein,
            "CellularComponent": "",
            "MolecularFunction": "",
            "BiologicalProcess": "",
            "PTM": "",
            "BiologicalProcess_Ancestors": "",
            "MolecularFunction_Ancestors": "",
        }

    # Step 1: extract GO terms & PTMs & GO Ids
    go_terms = extract_go_terms(result)
    ptm_terms = extract_ptm(result)
    go_ids = extract_go_ids(result)

    # Step 2: Retrieve ancestor GO IDs from QuickGO
    bp, mf = [], []

    with ThreadPoolExecutor(max_workers=8) as executor:
        future_to_go_id = {
            executor.submit(find_ancestors, go_id): go_id
            for go_id in go_ids
        }
        
        for future in as_completed(future_to_go_id):
            ancestors = future.result()
            root = find_root_ancestors(
                ancestors,
                biological_process_map,
                molecular_function_map
            )
            bp.extend(root["BiologicalProcess"])
            mf.extend(root["MolecularFunction"])

    return {
        "Uniprot_ID": protein,
        "CellularComponent": ", ".join(go_terms.get("C", [])),
        "MolecularFunction": ", ".join(go_terms.get("F", [])),
        "BiologicalProcess": ", ".join(go_terms.get("P", [])),
        "PTM": ", ".join(ptm_terms),
        "BiologicalProcess_Ancestors" : ", ".join(list(set(bp))),
        "MolecularFunction_Ancestors" : ", ".join(list(set(mf)))
    }


def safe_as_completed(futures, timeout=30):
    """Wrap as_completed to ensure no thread hangs forever."""
    for future in as_completed(futures, timeout=timeout):
        try:
            yield future.result()
        except TimeoutError:
            print("A thread took too long. Skipping...")
        except Exception as e:
            print(f"Error in thread: {e}")


def process_proteins_wrapper(
        protein_list,
        biological_process_map,
        molecular_function_map,
        num_workers=24
    ):
    """Main function to process proteins efficiently."""
    uniprot_data = {}

    # Step 1: Fetch UniProt data once per protein
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_protein = {
            executor.submit(fetch_uniprot_data, protein): protein
            for protein in protein_list
        }

        for future in tqdm(
            as_completed(future_to_protein),
            total=len(protein_list),
            desc="Fetching UniProt data"
        ):
            protein = future_to_protein[future]
            uniprot_data[protein] = future.result()

    # Step 2: Process each protein using stored data
    results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_protein = {
            executor.submit(
                process_protein,
                protein,
                uniprot_data,
                biological_process_map,
                molecular_function_map
            ): protein
            for protein in protein_list
        }

        for future in tqdm(
            as_completed(future_to_protein),
            total=len(protein_list),
            desc="Processing proteins"
        ):
            try:
                results.append(future.result())
            except Exception as e:
                pass

    # Step 3: Convert to DataFrame
    df = pd.DataFrame(results)

    df_annotations = df[[
        "Uniprot_ID",
        "CellularComponent",
        "MolecularFunction",
        "BiologicalProcess",
        "PTM"
    ]]
    df_ancestors = df[[
        "Uniprot_ID",
        "MolecularFunction_Ancestors",
        "BiologicalProcess_Ancestors"
    ]]
    
    return df_annotations, df_ancestors

In [ ]:
biological_process_map = {
    "GO:0016032": "viral process",
    "GO:0048519": "negative regulation of biological process",
    "GO:0048518": "positive regulation of biological process",
    "GO:0048511": "rhythmic process",
    "GO:0043473": "pigmentation",
    "GO:0065007": "biological regulation",
    "GO:0050789": "regulation of biological process",
    "GO:0098754": "detoxification",
    "GO:0051179": "localization",
    "GO:0002376": "immune system process",
    "GO:0051703":
        "biological process involved in intraspecies interaction between organisms",
    "GO:0040007": "growth",
    "GO:0040011": "locomotion",
    "GO:0042592": "homeostatic process",
    "GO:0044848": "biological phase",
    "GO:0050896": "response to stimulus",
    "GO:0022414": "reproductive process",
    "GO:0032502": "developmental process",
    "GO:0032501": "multicellular organismal process",
    "GO:0044419":
        "biological process involved in interspecies interaction between organisms",
    "GO:0009987": "cellular process"
}

molecular_function_map = {
    "GO:0003774": "cytoskeletal motor activity",
    "GO:0003824": "catalytic activity",
    "GO:0009055": "electron transfer activity",
    "GO:0016209": "antioxidant activity",
    "GO:0102910": "dirigent protein activity",
    "GO:0090729": "toxin activity",
    "GO:0065009": "regulation of molecular function",
    "GO:0098772": "molecular function regulator activity",
    "GO:0180051": "translation factor activity",
    "GO:0180024": "membrane grommet activity",
    "GO:0180020": "membrane bending activity",
    "GO:0005215": "transporter activity",
    "GO:0044868": "modulation by host of viral molecular function",
    "GO:0005198": "structural molecule activity",
    "GO:0140522": "fusogenic activity",
    "GO:0038024": "cargo receptor activity",
    "GO:0140911": "pore-forming activity",
    "GO:0140912": "membrane destabilizing activity",
    "GO:0140657": "ATP-dependent activity",
    "GO:0140691": "RNA folding chaperone",
    "GO:0140489": "molecular template activity",
    "GO:0005488": "binding",
    "GO:0140777": "protein-containing complex stabilizing activity",
    "GO:0140776": "protein-containing complex destabilizing activity",
    "GO:0045735": "nutrient reservoir activity",
    "GO:0060089": "molecular transducer activity",
    "GO:0060090": "molecular adaptor activity",
    "GO:0045182": "translation regulator activity",
    "GO:0141047": "molecular tag activity",
    "GO:0044093": "positive regulation of molecular function",
    "GO:0044092": "negative regulation of molecular function",
    "GO:0044183": "protein folding chaperone",
    "GO:0140223": "general transcription initiation factor activity",
    "GO:0140313": "molecular sequestering activity",
    "GO:0140110": "transcription regulator activity",
    "GO:0140104": "molecular carrier activity"
}

In [ ]:
all_data = {}
data_dir = "refactored_data"
csv_files = glob.glob(os.path.join(data_dir, '*.csv'))
for file in csv_files:
    key = os.path.splitext(os.path.basename(file))[0]
    df = pd.read_csv(file)
    all_data[key] = df

for k, v in all_data.items():
    print(k)
    display(df.head())

In [ ]:
protein_list = []
for k, v in all_data.items():
    protein_list.extend(v["Uniprot_ID"].to_list())
protein_list = list(set(protein_list))

print("\nProcessing all proteins:")

df_annotations, df_ancestor_annotations = process_proteins_wrapper(
    protein_list,
    biological_process_map,
    molecular_function_map,
    num_workers=24
)

In [ ]:
df_annotations.to_csv("go_annotations_detailed.csv", index=False)
df_annotations

In [ ]:
df_ancestor_annotations.to_csv("go_annotations_general.csv", index=False)
df_ancestor_annotations

## AR

In [ ]:
general = pd.read_csv("refactored_data/go_annotations_general.csv")
detailed = pd.read_csv("refactored_data/go_annotations_detailed.csv")

In [ ]:
for i, row in detailed.iterrows():
    if pd.notna(row["Cellular Component"]):
        row["Cellular Component"] = "{" + row["Cellular Component"] + "}"
    if pd.notna(row["Molecular Function"]):
        row["Molecular Function"] = "{" + row["Molecular Function"] + "}"
    if pd.notna(row["Biological Process"]):
        row["Biological Process"] = "{" + row["Biological Process"] + "}"
    if pd.notna(row["PTM"]):
        row["PTM"] = "{" + row["PTM"] + "}"

for i, row in general.iterrows():
    if pd.notna(row["Molecular Function"]):
        row["Molecular Function"] = "{" + row["Molecular Function"] + "}"
    if pd.notna(row["Biological Process"]):
        row["Biological Process"] = "{" + row["Biological Process"] + "}"

In [ ]:
detailed.to_csv("detailed_go.csv", index=False)
general.to_csv("general_go.csv", index=False)

# Analysis
### Plots

In [ ]:
csv_directory = 'refactored_data'
dataframes = {}
csv_files = [file_name
             for file_name
             in os.listdir(csv_directory)
             if file_name.endswith('.csv')]
for file_name in csv_files:
    if ("old" in file_name or "new" in file_name) and "supplementary" not in file_name and "ALL" not in file_name:
        file_path = os.path.join(csv_directory, file_name)
        df = pd.read_csv(file_path)
        name = file_path.replace(".csv", "").replace("refactored_data/", "")
        dataframes[name] = df

In [ ]:
go_detailed = pd.read_csv("go_annotations_detailed.csv")
go_general = pd.read_csv("go_annotations_general.csv")

In [ ]:
old_data_names = [name for name in dataframes.keys() if "old" in name]
old_data_counts = [len(df) for name, df in dataframes.items() if "old" in name]

fig_old = go.Figure(data=[go.Bar(
    x=old_data_names,
    y=old_data_counts,
    text=old_data_counts,
    textposition='auto',
    marker_color='orchid',
    marker_line_color='black',
    marker_line_width=2
)])

fig_old.update_layout(
    height=800,
    title=dict(
        text="Old Data",
        font=dict(size=40),
        automargin=True,
        yref='paper'
    ),
    xaxis_title="Dataset",
    yaxis_title="Entries",
    xaxis_tickangle=-45,
    showlegend=False
)

fig_old.show()

new_data_names = [name for name in dataframes.keys() if "new" in name]
new_data_counts = [len(df) for name, df in dataframes.items() if "new" in name]

fig_new = go.Figure(data=[go.Bar(
    x=new_data_names,
    y=new_data_counts,
    text=new_data_counts,
    textposition='auto',
    marker_color='sandybrown',
    marker_line_color='black',
    marker_line_width=2
)])

fig_new.update_layout(
    height=800,
    title=dict(
        text="New Data",
        font=dict(size=40),
        automargin=True,
        yref='paper'
    ),
    xaxis_title="Dataset",
    yaxis_title="Entries",
    xaxis_tickangle=-45,
    showlegend=False
)

fig_new.show()

In [ ]:
bps = []
mfs = []
ccs = []
ptms = []

for i, row in go_general.iterrows():
    bp_list_str = row["BiologicalProcess_Ancestors"]
    mf_list_str = row["MolecularFunction_Ancestors"]

    if not pd.isna(bp_list_str):
        bp_list = bp_list_str.split(", ")
        for bp in bp_list:
            if bp not in bps:
                bps.append(bp)

    if not pd.isna(mf_list_str):
        mf_list = mf_list_str.split(", ")
        for mf in mf_list:
            if mf not in mfs:
                mfs.append(mf)


for i, row in go_detailed.iterrows():
    cc_list_str = row["CellularComponent"]
    ptm_list_str = row["PTM"]

    if not pd.isna(cc_list_str):
        cc_list = cc_list_str.split(", ")
        for cc in cc_list:
            if cc not in ccs:
                ccs.append(cc)

    if not pd.isna(ptm_list_str):
        ptm_list = ptm_list_str.split(", ")
        for ptm in ptm_list:
            if ptm not in ptms:
                ptms.append(ptm)

In [ ]:
def plot_term_per_region(
        term,
        term_df,
        term_list,
        dataframes,
        old=True
):
    brain_regions_list = [
            "CB",
            "CC",
            "MB",
            "HC",
            "HT",
            "MD",
            "OB"
        ]

    brain_regions = {
        brain_region: {t: 0 for t in term_list}
        for brain_region in brain_regions_list
    }

    if term == "mf":
        go_term_title = "Molecular Function"
        par3 = "MolecularFunction_Ancestors"
    elif term == "bp":
        go_term_title = "Biological Process"
        par3 = "BiologicalProcess_Ancestors"
    elif term == "ptm":
        go_term_title = "Post-Translational Modifications"
        par3 = "PTM"
    else:
        raise ValueError(
            f"'term' must be either 'mf', 'bp' or 'ptm', not: '{term}'."
        )

    if old:
        for name, df in dataframes.items():
            if "old" in name:
                region = name.split("_")[1]
                for uID in df["Uniprot_ID"]:
                    if not term_df.loc[go_general["Uniprot_ID"] == uID].empty:
                        go_list_str = term_df.loc[
                            term_df["Uniprot_ID"] == uID,
                            par3
                        ].values[0]
                        if not pd.isna(go_list_str):
                            go_list = go_list_str.split(", ")
                            for go_term in go_list:
                                brain_regions[region][go_term] += 1
    else:
        for name, df in dataframes.items():
            if "new" in name:
                region = name.split("_")[1]
                for uID in df["Uniprot_ID"]:
                    if not term_df.loc[go_detailed["Uniprot_ID"] == uID].empty:
                        go_list_str = term_df.loc[
                            term_df["Uniprot_ID"] == uID,
                            par3
                        ].values[0]
                        if not pd.isna(go_list_str):
                            go_list = go_list_str.split(", ")
                            for go_term in go_list:
                                brain_regions[region][go_term] += 1

    categories = list(brain_regions.keys())
    subcategories = list(brain_regions["CB"].keys())

    # Grouped plot
    fig_grouped = go.Figure()

    for subcat in subcategories:
        fig_grouped.add_trace(go.Bar(
            x=categories,
            y=[brain_regions[cat][subcat] for cat in categories],
            name=subcat,
        ))

    fig_grouped.update_layout(
        barmode="group",
        height=1000,
        title=dict(
            text=f"Grouped {go_term_title} (GO) per brain region",
            font=dict(size=40),
            automargin=True,
            yref='paper'
        ),
        xaxis_title=dict(
            text="Brain part",
            font=dict(size=30)
        ),
        yaxis_title=dict(
            text="Counts",
            font=dict(size=30)
        ),
        legend_title=dict(
            text="GO terms",
            font=dict(size=20)
        ),
        legend=dict(
            font=dict(size=20),
            orientation="h",
            yanchor="top",
            xanchor="center",
            x=0.5
        ),
        xaxis=dict(
            tickfont=dict(size=20)
        ),
        yaxis=dict(
            tickfont=dict(size=20)
        )
    )
    fig_grouped.show()

    # Aggregate plot
    aggregate_by_term = {
        term: sum(brain_regions[region][term] for region in brain_regions_list)
        for term in subcategories
    }
    sorted_terms = sorted(
        aggregate_by_term.keys(),
        key=lambda x: aggregate_by_term[x])

    fig_aggregate = go.Figure()

    fig_aggregate.add_trace(go.Bar(
        x=sorted_terms,
        y=[aggregate_by_term[term] for term in sorted_terms],
        text=[aggregate_by_term[term] for term in sorted_terms],
        name="Aggregate by Term",
        marker_color='orchid',
        marker_line_color='black',
        marker_line_width=2
    ))
    fig_aggregate.update_layout(
        height=1000,
        title=dict(
            text=f"Aggregate {go_term_title} (GO) by Term (Sorted)",
            font=dict(size=40),
            automargin=True,
            yref='paper'
        ),
        xaxis_title=dict(
            text="GO Term",
            font=dict(size=30)
        ),
        yaxis_title=dict(
            text="Counts",
            font=dict(size=30)
        ),
        legend_title=dict(
            text="Aggregate by Term",
            font=dict(size=20)
        ),
        legend=dict(
            font=dict(size=20),
            orientation="h",
            yanchor="top",
            xanchor="center",
            x=0.5
        ),
        xaxis=dict(
            tickfont=dict(size=20)
        ),
        yaxis=dict(
            tickfont=dict(size=20)
        )
    )
    fig_aggregate.show()

    return brain_regions

### Old Dataset -> Taraslia

In [ ]:
_ = plot_term_per_region(
        "bp",
        go_general,
        bps,
        dataframes
)

In [ ]:
_ = plot_term_per_region(
        "mf",
        go_general,
        mfs,
        dataframes
)

In [ ]:
_ = plot_term_per_region(
        "ptm",
        go_detailed,
        ptms,
        dataframes
)

### New Dataset -> Korovesis 

In [ ]:
_ = plot_term_per_region(
        "bp",
        go_general,
        bps,
        dataframes,
        old=False
)

In [ ]:
_ = plot_term_per_region(
        "mf",
        go_general,
        mfs,
        dataframes,
        old=False
)

In [ ]:
_ = plot_term_per_region(
        "ptm",
        go_detailed,
        ptms,
        dataframes,
        old=False
)

### Brain regions 
#### Identifying unique & common proteins

In [ ]:
brain_part_df = pd.read_csv("refactored_data/brain_part.csv")
brain_part_df

In [ ]:
prot_per_brain_part = {}
for i, row in brain_part_df.iterrows():
    brain_part = row["Brain_part"]
    protein = row["Uniprot_ID"]
    if protein in prot_per_brain_part:
        prot_per_brain_part[protein].append(brain_part)
    else:
        prot_per_brain_part[protein] = [brain_part]

prot_per_brain_part

In [ ]:
proteins_in_one_brain_part = {
    prot: brain_parts[0]
    for prot, brain_parts
    in prot_per_brain_part.items()
    if len(brain_parts) == 1
    }

proteins_in_one_brain_part

In [ ]:
# Plot a bar plot a histogram of brain regions with counts of proteins with plotly

grouped_brain_parts = {}

for protein, brain_part in proteins_in_one_brain_part.items():
    if brain_part not in grouped_brain_parts:
        grouped_brain_parts[brain_part] = 1
    else:
        grouped_brain_parts[brain_part] += 1


fig_grouped_brain_parts = go.Figure()

fig_grouped_brain_parts.add_trace(go.Bar(
    x=list(grouped_brain_parts.keys()),
    y=list(grouped_brain_parts.values()),
    text=list(grouped_brain_parts.values()),
    name="Proteins in one brain part",
    marker_color='hotpink',
    marker_line_color='black',
    marker_line_width=2
))

fig_grouped_brain_parts.update_layout(
    height=800,
    title=dict(
        text="Proteins in one brain part",
        font=dict(size=40),
        automargin=True,
        yref='paper'
    ),
    xaxis_title=dict(
        text="Brain part",
        font=dict(size=30)
    ),
    yaxis_title=dict(
        text="Counts",
        font=dict(size=30)
    ),
    legend_title=dict(
        text="Proteins",
        font=dict(size=20)
    ),
    legend=dict(
        font=dict(size=20),
        orientation="h",
        yanchor="top",
        xanchor="center",
        x=0.5
    ),
    xaxis=dict(
        tickfont=dict(size=20)
    ),
    yaxis=dict(
        tickfont=dict(size=20)
    )
)

fig_grouped_brain_parts.show()


# Building a database 

In [ ]:
# Column names

new_data = [
    "Uniprot_ID",
    "Description",
    "Score",
    "Coverage",
    "# Proteins",
    "# Unique Peptides",
    "# Peptides",
    "# PSMs",
    "Area",
    "# AAs",
    "MW [kDa]",
    "calc. pI"
]

old_data = [
    "Uniprot_ID",
    "Protein name",
    "Mascot Score",
    "Coverage (%)",
    "Protein MW (Da)",
    "pI-Value"
]

new_supplementary = [
    "Uniprot_ID",
    "Entry name",
    "Protein names",
    "Cerebral cortex",
    "Olfactory Bulb",
    "Hippocampus",
    "Hypothalamus",
    "Mid brain",
    "Cerebellum",
    "Medulla"
]

old_supplementary = [
    "Uniprot_ID",
    "Protein name",
    "Protein MW",
    "pI-Value",
    "Brain part",
    "Subcellular localisation",
    "Function"
]


In [ ]:
main = [
    "Uniprot_ID",
    "Protein_Name",
    "# Proteins",
    "# Unique Peptides",
    "# Peptides",
    "# PSMs",
    "Area",
    "# AAs",
    "MW",
    "calc. pI"
]

brain_part_metrics = [
    "Uniprot_ID",
    "Brain_part",
    "Score",
    "Coverage",
]

go_terms_detailed = [
    "Uniprot_ID",
    "Biological_Process",
    "Molecular_Function",
    "Cellular_Component",
    "PTM"
]

go_terms_general = [
    "Uniprot_ID",
    "Biological_Process",
    "Molecular_Function"
]



In [ ]:
data_directory = "refactored_data/"

new_CB = pd.read_csv(data_directory + "new_CB.csv")
new_CC = pd.read_csv(data_directory + "new_CC.csv")
new_HC = pd.read_csv(data_directory + "new_HC.csv")
new_HT = pd.read_csv(data_directory + "new_HT.csv")
new_MB = pd.read_csv(data_directory + "new_MB.csv")
new_MD = pd.read_csv(data_directory + "new_MD.csv")
new_OB = pd.read_csv(data_directory + "new_OB.csv")

old_CB = pd.read_csv(data_directory + "old_CB.csv")
old_CC = pd.read_csv(data_directory + "old_CC.csv")
old_HC = pd.read_csv(data_directory + "old_HC.csv")
old_HT = pd.read_csv(data_directory + "old_HT.csv")
old_MB = pd.read_csv(data_directory + "old_MB.csv")
old_MD = pd.read_csv(data_directory + "old_MD.csv")
old_OB = pd.read_csv(data_directory + "old_OB.csv")

new_supplementary = pd.read_csv(data_directory + "new_supplementary.csv")
old_supplementary = pd.read_csv(data_directory + "old_ALL.csv")


all_data = {
    "new_CB": new_CB,
    "new_CC": new_CC,
    "new_HC": new_HC,
    "new_HT": new_HT,
    "new_MB": new_MB,
    "new_MD": new_MD,
    "new_OB": new_OB,
    "old_CB": old_CB,
    "old_CC": old_CC,
    "old_HC": old_HC,
    "old_HT": old_HT,
    "old_MB": old_MB,
    "old_MD": old_MD,
    "old_OB": old_OB,
    "new_supplementary": new_supplementary,
    "old_supplementary": old_supplementary
}

# Compare UniprotIDs of all dataframes
for key1 in all_data:
    for key2 in all_data:
        if key1 != key2:
            print(key1, key2)
            s1 = set(all_data[key1]["Uniprot_ID"])
            s2 = set(all_data[key2]["Uniprot_ID"])
            print(len(s1), len(s2), len(s1.intersection(s2)))
            # print(s1.intersection(s2))
            print()

In [ ]:
brain_part_names = {
    "CB": "Cerebellum",
    "CC": "Cerebral_Cortex",
    "HC": "Hippocampus",
    "HT": "Hypothalamus",
    "MB": "Mid_Brain",
    "MD": "Medulla",
    "OB": "Olfactory_Bulb"
}

for name, df in all_data.items():
    if name != "new_supplementary" and name != "old_supplementary":
        full_name = brain_part_names[name.split("_")[1]]
        df["Brain_part"] = full_name
        all_data[name] = df

for name, df in all_data.items():
    print(name)
    display(df.head())

In [ ]:
# Create new df with all Uniprot_ID and Brain_part
for name, df in all_data.items():
    if name != "new_supplementary" and name != "old_supplementary":
        all_data[name] = df[["Uniprot_ID", "Brain_part"]]


# use reduce to concat all dataframes except new_supplementary and old_supplementary
all_brain_parts = reduce(
    lambda x, y:
    pd.concat([x, y]),
    [all_data[key] for key in all_data
    if key != "new_supplementary" and key != "old_supplementary"]
)

# reset and drop index
all_brain_parts = all_brain_parts.reset_index(drop=True)

In [ ]:
# View the new dataframe
display(all_brain_parts)

In [ ]:
# For each protein in all_brain_parts, find proteins with more than one
# brain part and create a string with all brain parts separated by a comma
# so that each protein has only one row
all_brain_parts = all_brain_parts.groupby("Uniprot_ID")["Brain_part"].apply(
    lambda x: ', '.join(x)).reset_index()

# View the new dataframe
display(all_brain_parts)

In [ ]:
# Rename columns of new_supplementary to match the names in all_brain_parts
new_supplementary.rename(columns={
    "Cerebral cortex": "Cerebral_Cortex",
    "Olfactory Bulb": "Olfactory_Bulb",
    "Mid brain": "Mid_Brain",
}, inplace=True)
new_supplementary = new_supplementary[:-3]

In [ ]:
# Create a new column in new_supplementary with the brain part as a string separated
# by commas
new_supplementary["Brain_part"] = new_supplementary.apply(
    lambda row: (", ").join(row.index[row==1.0]), axis=1
    )
new_supplementary

In [ ]:
new_supplementary_subset = new_supplementary[["Uniprot_ID", "Brain_part"]]
new_supplementary_subset

In [ ]:
all_together = pd.concat([all_brain_parts, new_supplementary_subset])
all_together = all_together.reset_index(drop=True)
all_together.groupby("Uniprot_ID")["Brain_part"].apply(
    lambda x: ', '.join(x)).reset_index()
all_together

In [ ]:
all_brain_parts.to_csv(data_directory + "all_brain_parts.csv", index=False)

## Database

In [ ]:
data_directory = "refactored_data/"

new_CB = pd.read_csv(data_directory + "new_CB.csv")
new_CC = pd.read_csv(data_directory + "new_CC.csv")
new_HC = pd.read_csv(data_directory + "new_HC.csv")
new_HT = pd.read_csv(data_directory + "new_HT.csv")
new_MB = pd.read_csv(data_directory + "new_MB.csv")
new_MD = pd.read_csv(data_directory + "new_MD.csv")
new_OB = pd.read_csv(data_directory + "new_OB.csv")

old_CB = pd.read_csv(data_directory + "old_CB.csv")
old_CC = pd.read_csv(data_directory + "old_CC.csv")
old_HC = pd.read_csv(data_directory + "old_HC.csv")
old_HT = pd.read_csv(data_directory + "old_HT.csv")
old_MB = pd.read_csv(data_directory + "old_MB.csv")
old_MD = pd.read_csv(data_directory + "old_MD.csv")
old_OB = pd.read_csv(data_directory + "old_OB.csv")

new_supplementary = pd.read_csv(data_directory + "new_supplementary.csv")
old_supplementary = pd.read_csv(data_directory + "old_ALL.csv")

In [ ]:
all_data = {
    "new_CB": new_CB,
    "new_CC": new_CC,
    "new_HC": new_HC,
    "new_HT": new_HT,
    "new_MB": new_MB,
    "new_MD": new_MD,
    "new_OB": new_OB,
    "old_CB": old_CB,
    "old_CC": old_CC,
    "old_HC": old_HC,
    "old_HT": old_HT,
    "old_MB": old_MB,
    "old_MD": old_MD,
    "old_OB": old_OB
}

new_data = [
    "Uniprot_ID",
    "Description",
    "Score",
    "Coverage",
    "# Proteins",
    "# Unique Peptides",
    "# Peptides",
    "# PSMs",
    "Area",
    "# AAs",
    "MW [kDa]",
    "calc. pI"
]

old_data = [
    "Uniprot_ID",
    "Protein name",
    "Mascot Score",
    "Coverage (%)",
    "Protein MW (Da)",
    "pI-Value"
]

In [ ]:
for name, df in all_data.items():
    df.rename(
        columns={
            "Mascot Score": "Score",
            "Coverage (%)": "Coverage",
            "Protein MW (Da)": "mw",
            "Protein MW  (Da)": "mw",
            "pI-Value": "pI",
            "calc. pI": "pI",
            "# Proteins": "proteins",
            "# Unique Peptides": "unique_peptides",
            "# Peptides": "peptides",
            "# PSMs": "psms",
            "# AAs": "aas",
            "Description": "protein_name",
            "Protein names": "protein_name",
            "Protein name": "protein_name"
        }, inplace=True
    )
    full_name = brain_part_names[name.split("_")[1]]
    df["Brain_part"] = full_name
    all_data[name] = df
    if "old" in name:
        df["mw"] = df["mw"] / 1000
        all_data[name] = df
    if "new" in name:
        df["protein_name"] = df["protein_name"].str.split("OS=").str[0]
        all_data[name] = df
    

for name, df in all_data.items():
    display(df.head())

In [ ]:
new_CB = all_data["new_CB"]
new_CC = all_data["new_CC"]
new_HC = all_data["new_HC"]
new_HT = all_data["new_HT"]
new_MB = all_data["new_MB"]
new_MD = all_data["new_MD"]
new_OB = all_data["new_OB"]

old_CB = all_data["old_CB"]
old_CC = all_data["old_CC"]
old_HC = all_data["old_HC"]
old_HT = all_data["old_HT"]
old_MB = all_data["old_MB"]
old_MD = all_data["old_MD"]
old_OB = all_data["old_OB"]


main = pd.concat([
    new_CB,
    new_CC,
    new_HC,
    new_HT,
    new_MB,
    new_MD,
    new_OB,
    old_CB,
    old_CC,
    old_HC,
    old_HT,
    old_MB,
    old_MD,
    old_OB
    ])
main = main[[
    'Uniprot_ID',
    'protein_name'
]]

brain_part = pd.concat([
    new_CB,
    new_CC,
    new_HC,
    new_HT,
    new_MB,
    new_MD,
    new_OB,
    old_CB,
    old_CC,
    old_HC,
    old_HT,
    old_MB,
    old_MD,
    old_OB
    ])
brain_part = brain_part[[
    'Uniprot_ID',
    'Brain_part',
    'Score',
    'Coverage',
    'proteins',
    'unique_peptides',
    'peptides',
    'psms',
    'Area',
    'aas',
    'MW [kDa]',
    'pI'
]]


# drop duplicate uniprotid in main
main = main.drop_duplicates(subset=["Uniprot_ID"])

main.to_csv(data_directory + "main.csv", index=False)

brain_part.to_csv(data_directory + "brain_part.csv", index=False)



In [ ]:
detailed = pd.read_csv("go_annotations_detailed.csv")

# find and display  uniprot_ids in detailed df that are not in the main df
uniprot_ids = set(main["Uniprot_ID"])
detailed_uniprot_ids = set(detailed["Uniprot_ID"])
missing_uniprot_ids = detailed_uniprot_ids - uniprot_ids
missing_uniprot_ids = list(missing_uniprot_ids)
missing = detailed[detailed["Uniprot_ID"].isin(missing_uniprot_ids)]
display(len(missing))

# drop missing uniprot_ids from detailed df
detailed = detailed[~detailed["Uniprot_ID"].isin(missing_uniprot_ids)]
detailed.to_csv(data_directory + "go_annotations_detailed.csv", index=False)


# find and display uniptot_ids in general df that are not in the main df
general = pd.read_csv("go_annotations_general.csv")
uniprot_ids = set(main["Uniprot_ID"])
general_uniprot_ids = set(general["Uniprot_ID"])
missing_uniprot_ids = general_uniprot_ids - uniprot_ids
missing_uniprot_ids = list(missing_uniprot_ids)
missing = general[general["Uniprot_ID"].isin(missing_uniprot_ids)]
display(len(missing))

# drop missing uniprot_ids from general df
general = general[~general["Uniprot_ID"].isin(missing_uniprot_ids)]
general.to_csv(data_directory + "go_annotations_general.csv", index=False)


### Code for Database in SQL

In [ ]:
CREATE TABLE public.main (
    uniprot_id TEXT PRIMARY KEY,
    protein_name TEXT
);

create TYPE bp AS ENUM('Cerebellum', 'Cerebral_Cortex', 'Hippocampus', 'Hypothalamus', 'Mid_Brain', 'Medulla', 'Olfactory_Bulb');

CREATE TABLE public.brain_part (
    uniprot_id TEXT,
    brain_part bp,
    score FLOAT,
    coverage FLOAT,
    proteins INT,
    unique_peptides INT,
    peptides INT,
    psms INT,
    area FLOAT,
    aas INT,
    mw FLOAT,
    pi FLOAT,
    CONSTRAINT fk_brain_part FOREIGN KEY (uniprot_id) REFERENCES public.main (uniprot_id) ON DELETE CASCADE
);

CREATE TABLE public.go_terms_detailed (
    uniprot_id TEXT,
    biological_process TEXT[],
    molecular_function TEXT[],
    cellular_component TEXT[],
    ptm TEXT[],
    CONSTRAINT fk_go_terms_detailed FOREIGN KEY (uniprot_id) REFERENCES public.main (uniprot_id) ON DELETE CASCADE
);

CREATE TABLE public.go_terms_general (
    uniprot_id TEXT,
    biological_process TEXT[],
    molecular_function TEXT[],
    CONSTRAINT fk_go_terms_general FOREIGN KEY (uniprot_id) REFERENCES public.main (uniprot_id) ON DELETE CASCADE
);
